# KBMOD Demo

In [1]:
import os
import numpy as np

from pathlib import Path

from kbmod.fake_data_creator import *
from kbmod.run_search import *
from kbmod.search import *

# Setup file paths

There are at least two file paths you need to setup in order to run kbmod:
1. The im_filepath provides a path to the input images.
1. The res_filepath provides a path to the directory where the output results will be stored.

A time and psf file can optionally be specified.

If you already have data files, you can use those. Below we use the data in `data/demo`. You can also create your own fake data using `fake_data_creator.py`.

In [2]:
# Define the path for the data.
im_filepath = "../data/demo"
print(os.listdir(im_filepath))

['000000.fits', '000001.fits', '000002.fits', '000003.fits', '000004.fits', '000005.fits', '000006.fits', '000007.fits', '000008.fits', '000009.fits']


In [3]:
res_filepath = "./fake_results"
if not Path(res_filepath).is_dir():
    print(f"Directory {res_filepath} does not exist. Creating.")
    os.mkdir(res_filepath)

# Run KBMOD

In [4]:
results_suffix = "DEMO"

# The demo data has an object moving at x_v=10 px/day
# and y_v = 0 px/day. So we search velocities [0, 20].
v_min = 0
v_max = 20
v_steps = 21
v_arr = [v_min, v_max, v_steps]

# and angles [-0.5, 0.5]
ang_below = 0.5
ang_above = 0.5
ang_steps = 11
ang_arr = [ang_below, ang_above, ang_steps]

# There are 10 images in the demo data. Make sure we see
# the object in at least 7.
num_obs = 7

input_parameters = {
    # Required
    "im_filepath": im_filepath,
    "res_filepath": res_filepath,
    "time_file": None,
    "output_suffix": results_suffix,
    "v_arr": v_arr,
    "ang_arr": ang_arr,
    # Important
    "num_obs": num_obs,
    "do_mask": True,
    "lh_level": 10.0,
    "gpu_filter": True,
    # Fine tuning
    "sigmaG_lims": [15, 60],
    "mom_lims": [37.5, 37.5, 1.5, 1.0, 1.0],
    "peak_offset": [3.0, 3.0],
    "chunk_size": 1000000,
    "stamp_type": "cpp_median",
    "eps": 0.03,
    "clip_negative": True,
    "mask_num_images": 10,
    "cluster_type": "position",
    # Override the ecliptic angle for the demo data since we
    # know the true angle in pixel space.
    "average_angle": 0.0,
}

rs = SearchRunner()
rs.run_search_from_config(input_parameters)

---------------------------------------
Loading Images
---------------------------------------
Loaded 10 images
Times set
Starting Search
---------------------------------------
Ecliptic Angle = 1.1901
Min. Search Angle = -0.5000
Max Search Angle = 0.5000
Min Velocity = 0.0000
Max Velocity = 20.0000
Using in-line GPU sigmaG filtering methods
sigmaG limits: [15,60]
sigmaG coeff: 0.7753
Search finished in 0.057s
---------------------------------------
Retrieving Results
---------------------------------------
Getting results...
---------------------------------------
Chunk Start = 0
Chunk Max Likelihood = 243.35
Chunk Min. Likelihood = -1.00
---------------------------------------
Extracted batch of 1343 results for total of 1343
Applying Clipped-sigmaG Filtering
sigmaG limits: [15,60]
sigmaG coeff: 0.7753
0.18s elapsed
Completed filtering.
---------------------------------------
---------------------------------------
Applying Stamp Filtering
---------------------------------------
Stam

Check that results were written.

In [5]:
if os.path.exists(res_filepath):
    files = os.listdir(res_filepath)
    print(files)

['results_DEMO.txt', 'lc_DEMO.txt', 'psi_DEMO.txt', 'phi_DEMO.txt', 'lc_index_DEMO.txt', 'times_DEMO.txt', 'filtered_likes_DEMO.txt', 'ps_DEMO.txt', 'all_ps_DEMO.npy']
